### Building a Local Knowledge Assistant with LangChain & OpenAI

In this project, I implemented a `Retrieval-Augmented Generation (RAG)` pipeline using `LangChain`, `Chroma`, and `OpenAI embeddings`. The system retrieves relevant chunks of information from my document collection to provide more accurate and context-aware answers. By combining retrieval with generation, I can leverage a smaller, pre-trained LLM while still achieving detailed and precise responses, without the need for costly fine-tuning. This setup allows me to experiment with AI-driven Q&A in a practical, hands-on way.

This project was inspired by [RAG + Langchain Python Project: Easy AI/Chat For Your Docs](https://www.youtube.com/watch?v=tcqEUSNCn8I) and adapted for personal learning and experimentation.


### Why RAG Matters?

As language models grow larger, updating and controlling their knowledge becomes increasingly impractical through fine-tuning alone. `RAG` introduces a retrieval-based paradigm that preserves model generality while enabling precise, real-time knowledge integration. &nbsp;&nbsp; ⇾(Expand the section for details)

**Traditional Approaches** <br>
**`Fine-tuning`** works but comes with serious baggage: massive **compute requirements**, maintaining multiple **model versions**, knowledge that goes **stale** (requiring full retraining), and catastrophic **forgetting** where models lose general capabilities. **`LoRA`** makes this more efficient by training **only low-rank matrices** instead of full weights, but you're still **burning GPU hours**, tuning **hyperparameters**, and—critically—**baking knowledge** into opaque model weights. Want to update a single fact? Good luck auditing which training examples influenced what, or making changes without retraining from scratch.

**RAG: Knowledge Without Retraining** <br>
`RAG` flips the script. Instead of modifying the model, it retrieves **relevant context** from **vector databases at inference time**. The advantages are game-changing: **instant knowledge updates** without retraining, **traceable citations** grounding outputs in source material, and **zero training compute**—just embed and index your documents. Your knowledge stays **external** and **auditable**, perfect for rapidly changing domains like customer support or compliance where information updates weekly. Plus, you get clean separation of concerns: **the LLM handles reasoning while retrieval manages knowledge**.

**Trade-offs** <br>
The catch? You're now in the **retrieval** game. Chunking strategies, embedding quality, and similarity metrics directly impact what context reaches your model. Poor retrieval means **hallucinations**. **Context windows** limit how much you can include, requiring **smart ranking** when relevant info spans many documents. But these challenges beat managing expensive retraining pipelines—RAG trades GPU clusters for vector database tuning, a much more tractable problem.

### Why LangChain?

LLM applications in production need to integrate with dozens of **services—vector databases**, **document loaders**, **model providers**, **monitoring tools**. Alternatives like `Mirascope` or `LiteLLM` focus on **specific integration** challenges, but `LangChain`'s ecosystem is unmatched: **100+ integrations** maintained by both the core team and community, reducing custom boilerplate significantly.  &nbsp;&nbsp; ⇾(Expand the section for details)

**Going Solo: Reinventing the Wheel** <br>
**Going solo means building infrastructure from scratch**. `Raw API` calls seem simple until you're managing conversation history, implementing retry logic, handling streaming responses, and coordinating multi-step chains. **Custom frameworks give you control but demand serious engineering**: designing abstractions, integrating new model providers, building observability, maintaining compatibility. Every `RAG` pipeline or `agent system` requires **solving the same orchestration problems** teams everywhere have already tackled.

**What LangChain Brings to the Table** <br>
`LangChain` consolidates years of collective learning into **reusable patterns**. The framework shines in three areas: **provider abstraction** (switch between OpenAI, Anthropic, local models without rewriting logic), **composable chains** (LCEL makes complex workflows readable and debuggable), and **ecosystem integrations** (vector stores, document loaders, agent tools—all plug-and-play). You're not just getting code—you're getting architectural patterns that scale, from simple prompt chains to sophisticated agent loops with memory and tool use.

**What You're Trading For: Velocity vs Transparency** <br>
The tradeoff is **framework dependency**. `LangChain` adds abstraction layers that can **obscure what's happening under the hood**. Simple tasks might feel overengineered, and rapid framework evolution means staying current with breaking changes. Debugging requires understanding LangChain's execution model, not just your application logic. But the math usually works out: teams shipping complex LLM features fast versus teams stuck building plumbing. For most production use cases, LangChain's velocity wins outweigh the framework overhead.

### Why Chroma?

When working on LLM projects that involve `semantic search` or `RAG`, vector databases are essential for storing and querying embeddings efficiently. `Chroma` stands out because it **balances ease-of-use, flexibility, and functionality**. Unlike `FAISS` (a low-level similarity search library), `Pinecone` (managed cloud service), or `Weaviate` (complex self-hosted solution), `Chroma` lets you run embedded in **your application** or as a **standalone server**, supports metadata and persistence out-of-the-box, and integrates seamlessly with frameworks like LangChain—making it ideal for developers who want to move fast without infrastructure overhead. &nbsp;&nbsp; ⇾(Expand the section for details)

**The Vector Database Landscape** <br>
Many other solutions have their trade-offs. `FAISS` is extremely fast at large-scale similarity search but requires **manual management of metadata and persistence**. `Pinecone` offers a fully managed solution with advanced features but comes with **ongoing costs** and **cloud-only** limitations. `Weaviate` provides hybrid search and ML model integration but has a **steeper learning curve** and more **complex setup**. For engineers, these hurdles can slow down project development.

**Chroma: Simplicity Without Sacrifice** <br>
`Chroma`, on the other hand, simplifies the workflow. Its Python API is intuitive, it integrates naturally with `LangChain`, and it supports both **embedded** and **client-server deployment** modes. You can store embeddings along with metadata, filter results easily, and get your RAG pipelines running quickly. For small-to-medium scale projects, it allows you to focus on building intelligent LLM applications rather than spending time on database plumbing.

**Where Chroma Makes Trade-offs** <br>
Of course, `Chroma` has limitations. It **isn't optimized for extremely large datasets** with hundreds of millions of vectors and lacks some of the **advanced indexing options** that `FAISS` offers. However, for engineers who want to rapidly prototype and deploy LLM-powered solutions, `Chroma` provides a perfect balance of simplicity, power, and flexibility to get real results without unnecessary friction.


### Implementation

In [1]:
# Adapted from "RAG + Langchain Python Project: Easy AI/Chat For Your Docs"
# https://www.youtube.com/watch?v=tcqEUSNCn8I

from langchain_community.document_loaders import DirectoryLoader, PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_core.documents import Document
from langchain_openai import OpenAIEmbeddings
from langchain_openai import ChatOpenAI
from langchain_chroma import Chroma
from langchain_classic.prompts import ChatPromptTemplate
import openai 
from datasets import Dataset
from dotenv import load_dotenv
import os
import shutil
import numpy as np
import glob


d:\Program\GitHubProjects\Do-my-history-exam\test3.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


The project begins by **converting the target documents into LangChain** `Document` **objects** using the `DirectoryLoader` function. This preserves both the **text** and **metadata**, such as the source path and start index, which are required for LangChain's downstream functions.

In [2]:
# Convert the raw files into Document
def load_documents(DATA_PATH: str):
    documents = []
    for pdf_file in glob.glob(f"{DATA_PATH}/*.pdf"):
        loader = PyPDFLoader(pdf_file)
        docs = loader.load()
        documents.extend(docs)

    return documents

Next, the documents are **split into smaller chunks**. This is necessary primarily for **retrieval precision in RAG systems**. Smaller chunks allow the system to retrieve **only the most relevant information** rather than entire documents, improving semantic similarity matching. **Large chunks** can **dilute semantic meaning** and introduce **irrelevant context**, while **smaller chunks** enable more **focused retrieval**. Additionally, chunking helps manage **LLM context window limitations**, though retrieval quality is the primary consideration.

The 'RecursiveCharacterTextSplitter' controls how text is divided using two key parameters: 'chunk_size' and 'chunk_overlap'. 'chunk_size' defines the *maximum length* of each chunk, while 'chunk_overlap' *repeats a portion* of the previous chunk in the next one to **reduce the chance of splitting related information** across chunk boundaries. Choosing appropriate values for these parameters directly impacts retrieval quality, and thus the overall performance of the RAG system.

In [3]:
# Split the text into small chunks
def split_text(documents: list[Document], chunk_size: int=300, chunk_overlap: int=100):
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size,          #nb of characters in each chunk
        chunk_overlap=chunk_overlap,    #nb of characters to overlap between chunks
        length_function=len,            #decide how to measure the chunk, e.g., character, token, etc
        add_start_index=True,           #add the starting index of the chunk
    )
    chunks = text_splitter.split_documents(documents)
    print(f"Split {len(documents)} pages into {len(chunks)} chunks.\n")

    return chunks

Once the documents are split, each chunk is converted into a **vector embedding** using OpenAI's 'text-embedding-3-small' model. These **embeddings** are then stored in a 'Chroma' vector database along with the **original chunk text and metadata**. The 'Chroma' database enables efficient **similarity search** by **comparing query embeddings against stored chunk embeddings**.

It is crucial to use the **same embedding model** for both **indexing chunks** and **embedding queries** because different models produce **incompatible vector representations** that exist in different semantic spaces, making retrieval unreliable or impossible. For this project, I use the default embedding model `text-embedding-ada-002`.

In [4]:
# Apply vector embedding to chunks and save the embedding vector along with the content and metadata to database
def save_to_chroma(chunks: list[Document], CHROMA_PATH: str, model="text-embedding-ada-002"):
    # Clear out the database first.
    if os.path.exists(CHROMA_PATH):
        shutil.rmtree(CHROMA_PATH)

    db = Chroma.from_documents(
        documents=chunks, embedding=OpenAIEmbeddings(model=model), persist_directory=None
    )
    print(f"Saved {len(chunks)} chunks to {CHROMA_PATH}.")
    return db

Main

In [9]:
# Load environment variables that contains the OpenAI API key, LangChain can access it automatically
load_dotenv()

# Set path for where to get the original file and where to safe the chunks
CHROMA_PATH = "chroma"
DATA_PATH = "data/BoardGamesRuleBook"    # LLMs paper

# split the document in chunks and save it to database along with its embedded vector
documents = load_documents(DATA_PATH)
chunks = split_text(documents)

print(f"Print chunk 10 content: ")
document = chunks[10]
print(f"Content: \"{document.page_content}\"")
print(f"Metadata: {document.metadata}\n")

db = save_to_chroma(chunks, CHROMA_PATH)

Split 48 pages into 620 chunks.

Print chunk 10 content: 
Content: "3
CN3081CATAN5 of 5
v6.250401
LARGEST ARMY
LONGEST ROUTE
The first player to play 3 Knight cards receives this tile. If another player plays more Knight cards, they immediately receive this tile."
Metadata: {'producer': 'PyPDF', 'creator': 'PyPDF', 'creationdate': '2025-03-05T13:56:17-06:00', 'moddate': '2025-03-05T13:56:17-06:00', 'source': 'data/BoardGamesRuleBook\\CATAN.pdf', 'total_pages': 12, 'page': 2, 'page_label': '3', 'start_index': 0}

Saved 620 chunks to chroma.


test, pick the relevant paragraphs here

In [41]:
pdf_file = "data/BoardGamesRuleBook/CATAN.pdf"
loader = PyPDFLoader(pdf_file)
docs = loader.load()
    
print(docs[4].page_content)

5
© 2025 CATAN GmbH
© 2025 C
ATAN GmbH
© 2025 CATAN GmbH
© 2025 C
ATAN GmbH
© 2025 C
ATAN GmbH
© 2025 CATAN GmbH
© 2025 C
ATAN GmbH
© 2025 C
ATAN GmbH
© 2025 CATAN GmbH
© 2025 C
ATAN GmbH
© 2025 C
ATAN GmbH
© 2025 C
ATAN GmbH
5
6
4   Place the Robber and Player Pieces
• Place the robber on the desert hex.
• Each player selects a color and takes the roads and buildings 
(settlements and cities) in that color along with a player aid.  
In a 3-player game, do not use the white pieces.
• Place 2 starting settlements and roads for each player  
as shown.
5   Collect Y our Starting Resources 
Each player takes the resource cards from the supply that 
match the hexes adjacent to their second settlement, 
highlighted in black. Keep these cards hidden  
in your hand.
6   Choose the First Player 
Each player rolls the dice. The player with the 
highest roll is the first player.


In [43]:
# Choose a specific page and paragraph text
page_index = 4
target_paragraph = "Choose the First Player"

# Get the page text
page_text = docs[page_index].page_content

# Find where this paragraph appears
start_idx = page_text.find(target_paragraph)
if start_idx != -1:
    end_idx = start_idx + len(target_paragraph)
    print(f"Paragraph found on page {page_index + 1}")
    print(f"Start index: {start_idx}, End index: {end_idx}")
else:
    print("Paragraph not found in this page.")

Paragraph found on page 5
Start index: 773, End index: 796


The **prompt template** determines how **retrieved context** and the **user query** are **structured** for the **language model**. While prompt quality is difficult to quantify precisely, adhering to **established prompting principles significantly improves model responses**. Effective LLM prompting requires **clear, specific instructions** - similar to providing detailed directions to a new team member. Well-defined prompts guide the model toward desired outputs, while vague or ambiguous instructions increase output unpredictability, often resulting in irrelevant or inaccurate responses.

In [6]:
PROMPT_TEMPLATE = """
Answer the question based only on the following context:

{context}

---

Answer the question based on the above context: {question}
"""

Once the Chroma database and prompt template are configured, the system can process user queries through the following pipeline: First, the query is embedded using the **same embedding model** applied to the document chunks, ensuring vector space consistency. The system then retrieves the **top-k most similar chunks** from the vector database (using L2 (Euclidean) distance by default). These retrieved chunks are combined with the original query according to the prompt template structure and passed to the LLM (default LLM: `gpt-3.5-turbo`) for response generation.

Implementing **quality safeguards** is essential for production systems. This includes **rejecting empty** or **malformed queries** and filtering results when similarity scores fall **below a confidence threshold**, as low-similarity retrievals typically indicate insufficient relevant context and lead to unreliable outputs. With these components in place, the RAG pipeline can effectively retrieve pertinent information and generate well-informed responses.

In [5]:
# Generate the answer by feeding the LLM with prompt
def generate_answer(question: str, context: str, isprintprompt: bool=False) :
    # Generate the prompt template with context and query
    context_text = "\n\n---\n\n".join([doc.page_content for doc, _score in context])
    prompt_template = ChatPromptTemplate.from_template(PROMPT_TEMPLATE)
    prompt = prompt_template.format(context=context_text, question=question)
    if isprintprompt:
        print(prompt)

    # Implement the LLM and feed it with the prompt
    model = ChatOpenAI(model="gpt-3.5-turbo")
    return model.invoke(prompt) 

Main

In [8]:
# query_text = input("Enter your query: ")
query_text = "Can you steal from a city or settlement you just built this turn?"

# Search the DB.
results = db.similarity_search_with_relevance_scores(query_text, k=3)
if len(results) == 0 or results[0][1] < 0.7:
    print(f"Unable to find matching results, similarity too low.")
    # return

print("Top 3 similarity:")
for i, k in enumerate(results):
    if i == 3:
        break
    print(f"Top {i + 1}:\nContent: {k[0].page_content}\nL2 similarity: {k[1]}\n")

response_text = generate_answer(query_text, results, True)

# Print the formatted response
sources = [doc.metadata.get("source", None) for doc, _score in results]
formatted_response = f"Response: {response_text.content}\nSources: {sources}"
print(formatted_response)


Top 3 similarity:
Top 1:
Content: does not move, and you may not steal a card from another player.
Once the robber is placed on the board, it activates as usual. You must move 
it to a new hex and steal 1 random card from the hand (resource cards + 
commodity cards) of a player who has a building on that hex.
L2 similarity: 0.7823288076491218

Top 2:
Content: If a player has more than one building on the new hex, only steal 1 card. You may only play this 
card after the first barbarian attack (when the robber is placed on the board).
© 2025 CATAN GmbH
VICTORY POINT
CONSTITUTION
Play immediately into your  
player area, even if it is  
not your turn.
L2 similarity: 0.7805705254229444

Top 3:
Content: You may not move the pirate until you have at least one route 
between one of your buildings and a village on a small island. When 
you move the pirate, you may either steal a resource card or a cloth 
token from a player whose ship is on an edge of the pirate’s new hex.
WINNING THE GAME
L2

### Evaluation
`RAG` evaluation breaks down into two critical stages: **retrieval** and **generation**.

**Retrieval Evaluation**<br>
In retrieval evaluation, we **measure how effectively the system identifies and ranks relevant documents for a given query**. Standard metrics like `Precision@k`, `Recall@k`, and `Mean Reciprocal Rank (MRR)` quantify whether the retriever surfaces useful chunks in its top results. High precision means most retrieved chunks are relevant; high recall means we're not missing critical information. These metrics guide optimization of chunk size, overlap, and embedding model selection—essentially tuning what context reaches the generator.

**Generation Evaluation**<br>
Generation evaluation **assesses the quality and factual grounding of responses given the retrieved context**. We use `RAGAS (Retrieval-Augmented Generation Assessment)` metrics: `faithfulness` (does the answer align with retrieved evidence?), answer correctness (how close to ground truth?), `answer relevancy` (is it on-topic?), and `context precision/recall` (did we retrieve the right supporting passages?). These metrics reveal not just what the model generates, but how well it uses retrieved knowledge.

In [6]:
import json
from typing import List, Dict

# Extract content from Q&A file generated with ChatGPT
def read_qa_file(file_path: str) -> List[Dict]:
    """
    Reads a minimal JSON file for Q&A evaluation.
    
    Expected format:
    [
        {
            "question": "...",
            "relevant_chunks": ["text1", "text2"],
            "answer": "..."
        }
    ]
    """
    with open(file_path, "r", encoding="utf-8") as f:
        data = json.load(f)

    rows = []
    for item in data:
        rows.append({
            "question": item["question"],
            "reference": item["answer"],
            "relevant_chunk_texts": item["relevant_chunks"],
        })
    return rows

In [ ]:
from difflib import SequenceMatcher

def text_overlap_ratio(retrieved_text: str, ground_truth_text: str) -> float:
    """
    Compute how much of the ground truth text is contained in the retrieved text.
    
    Args:
        retrieved_text: Text from retrieved chunk (typically longer)
        ground_truth_text: Text from ground truth chunk (what we're looking for)
        method: "word" for word-based, "char" for character-based, "jaccard" for Jaccard similarity
        
    Returns:
        Ratio of ground truth covered by retrieved text (0 to 1)
        - 1.0 means GT is fully contained in retrieved
        - 0.5 means half of GT words are in retrieved
        - 0.0 means no overlap
    """
    # Normalize: lowercase and remove extra whitespace
    retrieved_normalized = ' '.join(retrieved_text.lower().split())
    gt_normalized = ' '.join(ground_truth_text.lower().split())
    
    if len(gt_normalized) == 0:
        return 0.0
    
    # Check if GT appears as a substring (consecutive words)
    if gt_normalized in retrieved_normalized:
        return 1.0
    
    # Check for longest common substring ratio
    matcher = SequenceMatcher(None, gt_normalized, retrieved_normalized)
    match = matcher.find_longest_match(0, len(gt_normalized), 0, len(retrieved_normalized))
    
    # Return ratio of longest consecutive match
    return match.size / len(gt_normalized)

In [ ]:
# # Load environment variables that contains the OpenAI API key, LangChain can access it automatically
# load_dotenv()

# # Set path for where to get the original file and where to safe the chunks
# CHROMA_PATH = "chroma"
# DATA_PATH = "data/BoardGamesRuleBook"    # LLMs paper    
qa_file = "data/BoardGamesRuleBook/CATAN_train.json"

# chunks = split_text(documents, 300, 50)
# # db = save_to_chroma(chunks, CHROMA_PATH, "text-embedding-3-small")
# db = save_to_chroma(chunks, CHROMA_PATH, "text-embedding-ada-002")


path = qa_file
chroma = db
k = 3
overlap_threshold: float = 0.8,

evaluation_rows = []

# Read JSON Q&A file
qa_rows = read_qa_file(path)

for row in qa_rows:
    question = row["question"]
    ground_truth = row["reference"]
    ground_truth_relevant_texts = row["relevant_chunk_texts"]

    # Retrieve context from database (with current chunking config)
    context_results = chroma.similarity_search_with_relevance_scores(question, k=k)
    retrieved_texts = [doc.page_content for doc, _score in context_results]
    retrieved_scores = [score for _doc, score in context_results]
    # TEXT-BASED MATCHING: Check which retrieved chunks overlap with ground truth
    relevant_retrieved_indices = []
    for i, retrieved_text in enumerate(retrieved_texts):
        for gt_text in ground_truth_relevant_texts:
            overlap = text_overlap_ratio(retrieved_text, gt_text)

            print(f"Question: {question}\nRetrieved {i}: \n{retrieved_text}\nGT: \n{gt_text}\nOverlap: \n{overlap}\nScore: {retrieved_scores[i]}\n")
            # if overlap >= overlap_threshold:
            #     relevant_retrieved_indices.append(i)
            #     break  # This retrieved chunk matches ground truth, move to next

    # # Use indices as pseudo-IDs for metric computation
    # retrieved_chunk_ids = list(range(len(retrieved_texts)))
    # relevant_chunk_ids = relevant_retrieved_indices

    # # Generate answer using LLM
    # answer = generate_answer(question, context_results)

    # evaluation_rows.append({
    #     "question": question,
    #     "contexts": retrieved_texts,
    #     "retrieved_chunk_ids": retrieved_chunk_ids,  # [0, 1, 2, ...]
    #     "relevant_chunk_ids": relevant_chunk_ids,    # [0, 2] if chunks 0 and 2 match GT
    #     "answer": answer.content if hasattr(answer, 'content') else str(answer),
    #     "ground_truth": ground_truth,
    #     "source": row.get("board_game", "unknown")

Question: Can you play a development card the same turn you buy it?
Retrieved 0: 
You may play 1 development card during your turn by placing it face up in your player area.  
It may not be a card you built this turn. You may play a development card before rolling dice 
or at any time during the Action phase.
GT: 
Not on the turn you bought it
Overlap: 
0.10344827586206896
Score: 0.8566725820415793

Question: Can you play a development card the same turn you buy it?
Retrieved 1: 
PLAY DEVELOPMENT CARDS
• Not on the turn you bought it
• Only 1 development card per turn
• Before rolling dice or during the Action phase
VP card exception: You may play multiple VP cards (even on the 
turn you buy them) in order to win the game.
CN3081
CATAN
3 of 5
v6.250401
VP0
VP1
VPs2
VPs?
VP0
GT: 
Not on the turn you bought it
Overlap: 
1.0
Score: 0.8393287640209977

Question: Can you play a development card the same turn you buy it?
Retrieved 2: 
PLAY DEVELOPMENT CARDS
• Not on the turn you bought it
• 

In [ ]:
def generate_answer_in_eval_format(
    path: str, 
    chroma, 
    k: int,
    overlap_threshold: float = 0.5,
    overlap_method: str = "word"
) -> List[Dict]:
    """
    Generates answers in evaluation format compatible with RAGas and Hugging Face metrics.
    Returns a list of dicts containing:
        - question
        - contexts (retrieved text)
        - retrieved_chunk_ids (all retrieved)
        - relevant_chunk_ids (ground truth relevant)
        - answer (generated by LLM)
        - reference (ground truth)
        - source (board game / paper)
    """
    evaluation_rows = []

    # Read JSON Q&A file
    qa_rows = read_qa_file(path)

    for row in qa_rows:
        question = row["question"]
        ground_truth = row["reference"]
        ground_truth_relevant_texts = row["relevant_chunk_texts"]

        # Retrieve context from database (with current chunking config)
        context_results = chroma.similarity_search_with_relevance_scores(question, k=k)
        retrieved_texts = [doc.page_content for doc, _score in context_results]

        # TEXT-BASED MATCHING: Check which retrieved chunks overlap with ground truth
        relevant_retrieved_indices = []
        for i, retrieved_text in enumerate(retrieved_texts):
            for gt_text in ground_truth_relevant_texts:
                overlap = text_overlap_ratio(retrieved_text, gt_text, method=overlap_method)
                if overlap >= overlap_threshold:
                    relevant_retrieved_indices.append(i)
                    break  # This retrieved chunk matches ground truth, move to next

        # Use indices as pseudo-IDs for metric computation
        retrieved_chunk_ids = list(range(len(retrieved_texts)))
        relevant_chunk_ids = relevant_retrieved_indices

        # Generate answer using LLM
        answer = generate_answer(question, context_results)

        evaluation_rows.append({
            "question": question,
            "contexts": retrieved_texts,
            "retrieved_chunk_ids": retrieved_chunk_ids,  # [0, 1, 2, ...]
            "relevant_chunk_ids": relevant_chunk_ids,    # [0, 2] if chunks 0 and 2 match GT
            "answer": answer.content if hasattr(answer, 'content') else str(answer),
            "ground_truth": ground_truth,
            "source": row.get("board_game", "unknown")
        })

    return evaluation_rows


Main

In [37]:
# generate answer in evaluation format
# qa_file = "data/papers/Beginner Q&A.txt"
qa_file = "data/BoardGamesRuleBook/boardgame_retrieval_dataset.json"
evaluation_rows = generate_answer_in_eval_format(path=qa_file, chroma=db, k=2)

# change it to Dataset format
evaluation_dataset = Dataset.from_list(evaluation_rows)

In [ ]:
import itertools
import pandas as pd
import numpy as np
from typing import List
from datasets import Dataset
import evaluate

# --- Retrieval Evaluation ---
def run_retrieval_evaluation(
    documents: List[str],
    qa_file: str,
    chunk_sizes: List[int],
    chunk_overlaps: List[int],
    embedding_models: List[str],
    top_k_list: List[int],
    chroma_path: str = "chroma"
    ) -> pd.DataFrame:
    """
    Runs Hugging Face retrieval evaluation (Precision@K, Recall@K, MRR, NDCG@K, Hit Rate@K).
    
    Parameters:
        documents: List of raw document strings to split and embed.
        qa_file: Path to JSON Q&A file for evaluation.
        chunk_sizes: List of chunk sizes to test.
        chunk_overlaps: List of overlap sizes to test.
        embedding_models: List of embedding models to test.
        top_k_list: List of top_k values to test for retrieval.
        chroma_path: Directory to save Chroma embeddings.
        
    Returns:
        DataFrame with retrieval metrics for all tested configurations.
    """

    # Initialize HF metrics
    
    hf_metrics = {
        "precision": evaluate.load("precision", module_type="metric"),
        "recall": evaluate.load("recall", module_type="metric"),
        # "mrr": evaluate.load("mrr", module_type="metric"),
        # NDCG is provided via "ndcg" metric if using module_type="metric" with BEIR installed
        # "ndcg": evaluate.load("ndcg", module_type="metric"),  
        # Hit rate is not directly available, we can use recall@k as a proxy or compute manually
    }

    results = []

    for chunk_size, overlap, emb_model, top_k in itertools.product(
        chunk_sizes, chunk_overlaps, embedding_models, top_k_list
    ):
        print(f"\n🔧 Retrieval config: chunk={chunk_size}, overlap={overlap}, model={emb_model}, top_k={top_k}")
        chunks = split_text(documents, chunk_size, overlap)
        db = save_to_chroma(chunks, chroma_path, emb_model)
        evaluation_rows = generate_answer_in_eval_format(
                qa_file, db, top_k
            )

        predictions = [row["retrieved_chunk_ids"] for row in evaluation_rows]
        references = [row["relevant_chunk_ids"] for row in evaluation_rows]


        hf_scores = {name: metric.compute(predictions=predictions, references=references, k=top_k)
                     for name, metric in hf_metrics.items()}

        results.append({
            "chunk_size": chunk_size,
            "chunk_overlap": overlap,
            "embedding_model": emb_model,
            "top_k": top_k,
            "precision_at_k": hf_scores["precision"]["precision"],
            "recall_at_k": hf_scores["recall"]["recall"],
            "mrr_at_k": hf_scores["mrr"]["mrr"],
            "ndcg_at_k": hf_scores["ndcg"]["ndcg"],
            "hit_rate_at_k": hf_scores["hit_rate"]["hit_rate"],
        })

    df = pd.DataFrame(results)
    df.to_csv("retrieval_eval_results.csv", index=False)
    print("\n✅ Retrieval evaluation complete!")
    return df


# # --- Generation Evaluation ---
# def run_generation_evaluation(documents: List[str], qa_file: str, chunk_size: int, overlap: int, 
#                               embedding_model: str, top_k: int, chroma_path: str = "chroma") -> pd.DataFrame:
#     """
#     Runs RAGas generation evaluation (answer correctness, relevancy, faithfulness, context precision/recall).
    
#     Parameters:
#         documents: List of raw document strings to split and embed.
#         qa_file: Path to JSON Q&A file for evaluation.
#         chunk_size: Chunk size for splitting documents.
#         overlap: Overlap size for splitting documents.
#         embedding_model: Embedding model name.
#         top_k: Number of top chunks to retrieve for LLM context.
#         chroma_path: Directory to save Chroma embeddings.
        
#     Returns:
#         DataFrame with RAGas generation metrics.
#     """
#     from ragas import evaluate
#     from ragas.metrics import (
#         answer_correctness,
#         answer_relevancy,
#         faithfulness,
#         context_precision,
#         context_recall,
#     )
#     from openai import ChatOpenAI

#     chunks = split_text(documents, chunk_size, overlap)
#     db = save_to_chroma(chunks, chroma_path, embedding_model)
#     evaluation_rows = generate_answer_in_eval_format(qa_file, db, top_k)
#     evaluation_dataset = Dataset.from_list(evaluation_rows)

#     llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)

#     # --- Run RAGas evaluation ---
#     scores = evaluate(
#         evaluation_dataset,
#         metrics=[
#             answer_correctness,
#             answer_relevancy,
#             faithfulness,
#             context_precision,
#             context_recall,
#         ],
#         llm=llm,
#     )

#     # Save results
#     df = pd.DataFrame([{
#         "chunk_size": chunk_size,
#         "chunk_overlap": overlap,
#         "embedding_model": embedding_model,
#         "top_k": top_k,
#         "answer_correctness_mean": np.mean(scores["answer_correctness"]),
#         "answer_relevancy_mean": np.mean(scores["answer_relevancy"]),
#         "faithfulness_mean": np.mean(scores["faithfulness"]),
#         "context_precision_mean": np.mean(scores["context_precision"]),
#         "context_recall_mean": np.mean(scores["context_recall"]),
#     }])
    
#     df.to_csv("generation_eval_results.csv", index=False)
#     print("\n✅ Generation evaluation complete!")
#     return df

In [59]:
    
qa_file = "data/BoardGamesRuleBook/CATAN_train.json"

retrieval_df = run_retrieval_evaluation(
    documents=documents,
    qa_file=qa_file,
    chunk_sizes=[300, 500],
    chunk_overlaps=[50, 100],
    embedding_models=["text-embedding-3-small"],
    top_k_list=[2, 5, 10],
    split_text_func=split_text,
    save_to_chroma_func=save_to_chroma,
    generate_answer_func=generate_answer
)

display(retrieval_df)



🔧 Retrieval config: chunk=300, overlap=50, model=text-embedding-3-small, top_k=2
Split 48 pages into 510 chunks.

Saved 510 chunks to chroma.


ValueError: Predictions and/or references don't match the expected format.
Expected format: {'predictions': Value('int32'), 'references': Value('int32')},
Input predictions: [[0, 1], [0, 1], [0, 1], ..., [0, 1], [0, 1], [0, 1]],
Input references: [[], [], [], ..., [], [], []]

**Why RAGAS?** <br>
Traditional NLP metrics like `BLEU` or `ROUGE` measure surface-level text similarity but ignore whether answers are grounded in evidence—critical for `RAG` systems where factual accuracy and traceability matter. `RAGAS` provides task-aware, automated evaluation that directly **measures how generated content connects to retrieved context**. Its modular design integrates seamlessly with `LangChain` and `Weights & Biases`, enabling scalable experimentation and real-time tracking. Essentially, `RAGAS` lets us **evaluate retrieval-generation alignment** systematically, leading to more interpretable and reliable `RAG` performance insights without constant human annotation.

**The limitations?** <br>
`RAGAS` relies on **LLM-as-judge evaluation**, meaning it **uses language models (often GPT-4) to score metrics** like `faithfulness` and `relevancy`. This introduces **cost** (API calls for every evaluation), **latency** (slower than traditional metrics), and **potential bias** (the judge model's own limitations affect scores). Additionally, `RAGAS` metrics **require ground truth datasets** for answer correctness, which aren't always available or easy to create for domain-specific applications. The framework also has a **learning curve**—understanding what each metric measures and how to interpret scores together requires familiarity with `RAG`-specific evaluation concepts. Despite these trade-offs, for most `RAG` applications, `RAGAS`'s ability to measure grounding and factuality outweighs the operational overhead compared to purely manual evaluation or metrics that ignore retrieval quality entirely.

In [29]:
import numpy as np
from ragas import evaluate
from ragas.metrics import (
    answer_correctness,
    answer_relevancy,
    faithfulness,
    context_precision,
    context_recall,
)

# Initialize the LLM for evaluation
llm = ChatOpenAI(model="gpt-3.5-turbo")  # sync interface compatible with RAGas

# Run evaluation
scores = evaluate(
    evaluation_dataset,
    metrics=[
        answer_correctness,
        answer_relevancy,
        faithfulness,
        context_precision,
        context_recall,
    ],
    llm=llm,  # pass the LLM explicitly
)

# Print the results
print("Evaluation dataset preview:")
for i, row in enumerate(evaluation_dataset):
    if i >= 5: #only print the first 5 Q&A result
        break
    print(f"{i+1}. Q: {row['question']}")
    print(f"   Answer: {row['answer']}")
    print(f"   Reference: {row['reference']}")
    print(f"   Contexts: {row['contexts']}\n")

Evaluating:  52%|█████▏    | 200/385 [09:59<05:49,  1.89s/it]LLM returned 1 generations instead of requested 3. Proceeding with 1 generations.
LLM returned 1 generations instead of requested 3. Proceeding with 1 generations.
Evaluating:  63%|██████▎   | 244/385 [12:17<07:44,  3.29s/it]LLM returned 1 generations instead of requested 3. Proceeding with 1 generations.
LLM returned 1 generations instead of requested 3. Proceeding with 1 generations.
Evaluating:  88%|████████▊ | 337/385 [16:46<02:19,  2.91s/it]LLM returned 1 generations instead of requested 3. Proceeding with 1 generations.
LLM returned 1 generations instead of requested 3. Proceeding with 1 generations.
Evaluating: 100%|██████████| 385/385 [19:02<00:00,  2.97s/it]


Evaluation dataset preview:
1. Q: What is needed to buy a development card?
   Answer: To buy a development card, you need to build or move one of your ships to an edge with a development card.
   Reference: 1 wool, 1 grain, and 1 ore.
   Contexts: ['DEVELOPMENT CARDS\nWhen you build or move one of your ships to an edge with a development card, \ntake the card. Use it as if you bought it this turn (i.e., you may play it on a future \nturn or immediately reveal it to win the game).\nPORTS', 'You may play 1 development card during your turn by placing it face up in your player area.  \nIt may not be a card you built this turn. You may play a development card before rolling dice \nor at any time during the Action phase.']

2. Q: Can you play multiple development cards on the same turn?
   Answer: No, you can only play one development card per turn according to the context provided.
   Reference: No, only one development card per turn (except Victory Points).
   Contexts: ['PLAY DEVELOPMEN

In [30]:
# print RAGAS result
print(
    f"RAGAS scores:"
    f"\n Answer correctness: mean={np.mean(scores['answer_correctness']):.4f}, std={np.std(scores['answer_correctness']):.4f}"
    f"\n Answer relevancy:   mean={np.mean(scores['answer_relevancy']):.4f}, std={np.std(scores['answer_relevancy']):.4f}"
    f"\n Faithfulness:       mean={np.mean(scores['faithfulness']):.4f}, std={np.std(scores['faithfulness']):.4f}"
    f"\n Context precision:  mean={np.mean(scores['context_precision']):.4f}, std={np.std(scores['context_precision']):.4f}"
    f"\n Context recall:     mean={np.mean(scores['context_recall']):.4f}, std={np.std(scores['context_recall']):.4f}"
)

RAGAS scores:
 Answer correctness: mean=0.4701, std=0.2603
 Answer relevancy:   mean=0.9503, std=0.1192
 Faithfulness:       mean=0.4361, std=0.3741
 Context precision:  mean=0.6948, std=0.4199
 Context recall:     mean=0.7403, std=0.4385


In [32]:
import itertools
import numpy as np
import pandas as pd
from ragas import evaluate
from ragas.metrics import (
    answer_correctness,
    answer_relevancy,
    faithfulness,
    context_precision,
    context_recall,
)
from datasets import Dataset
from IPython.display import display


# Load environment variables that contains the OpenAI API key, LangChain can access it automatically
load_dotenv()

# Set path for where to get the original file and where to safe the chunks
CHROMA_PATH = "chroma"
DATA_PATH = "data/BoardGamesRuleBook"    # LLMs paper

# Example parameter grid
# chunk_sizes = [256, 512, 1024]
# chunk_overlaps = [0, 50]
# embedding_models = ["text-embedding-3-small", "text-embedding-3-large"]
# retriever_top_k = [2, 5]

chunk_sizes = [300]
chunk_overlaps = [0, 50, 100]
embedding_models = ["text-embedding-3-small"]
retriever_top_k = [2]

results = []
documents = load_documents(DATA_PATH)

for chunk_size, overlap, emb_model, top_k in itertools.product(chunk_sizes, chunk_overlaps, embedding_models, retriever_top_k):
    print(f"\n🔧 Running config: chunk={chunk_size}, overlap={overlap}, model={emb_model}, top_k={top_k}")

    # --- Step 1: Build your retriever & RAG pipeline ---
    chunks = split_text(documents, chunk_size, overlap)
    db = save_to_chroma(chunks, CHROMA_PATH, emb_model)

    # generate answer in evaluation format and change it to dataset
    qa_file = "data/BoardGamesRuleBook/CATAN_Q&A_Eval.txt"
    evaluation_rows = generate_answer_in_eval_format(qa_file, db, top_k)
    evaluation_dataset = Dataset.from_list(evaluation_rows)

    # sync interface compatible with RAGas
    llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)  # Explicitly pass to model kwargs)  

    # Run evaluation
    scores = evaluate(
        evaluation_dataset,
        metrics=[
            answer_correctness,
            answer_relevancy,
            faithfulness,
            context_precision,
            context_recall,
        ],
        llm=llm,  # pass the LLM explicitly
    )

    # --- Step 3: Collect results ---
    results.append({
        "chunk_size": chunk_size,
        "chunk_overlap": overlap,
        "embedding_model": emb_model,
        "top_k": top_k,
        "answer_correctness_mean": np.mean(scores["answer_correctness"]),
        "answer_correctness_std": np.std(scores["answer_correctness"]),
        "answer_relevancy_mean": np.mean(scores["answer_relevancy"]),
        "answer_relevancy_std": np.std(scores["answer_relevancy"]),
        "faithfulness_mean": np.mean(scores["faithfulness"]),
        "faithfulness_std": np.std(scores["faithfulness"]),
        "context_precision_mean": np.mean(scores["context_precision"]),
        "context_precision_std": np.std(scores["context_precision"]),
        "context_recall_mean": np.mean(scores["context_recall"]),
        "context_recall_std": np.std(scores["context_recall"]),
    })

# --- Step 4: Save and inspect ---
df = pd.DataFrame(results)
df.to_csv("ragas_auto_eval_results.csv", index=False)
print("\n✅ Evaluation complete! Results saved to ragas_auto_eval_results.csv")
display(df)



🔧 Running config: chunk=300, overlap=0, model=text-embedding-3-small, top_k=2
Split 48 pages into 469 chunks.

Saved 469 chunks to chroma.


Evaluating:   0%|          | 0/50 [00:00<?, ?it/s]LLM returned 1 generations instead of requested 3. Proceeding with 1 generations.
LLM returned 1 generations instead of requested 3. Proceeding with 1 generations.
LLM returned 1 generations instead of requested 3. Proceeding with 1 generations.
Evaluating: 100%|██████████| 50/50 [02:30<00:00,  3.01s/it]



🔧 Running config: chunk=300, overlap=50, model=text-embedding-3-small, top_k=2
Split 48 pages into 510 chunks.

Saved 510 chunks to chroma.


Evaluating:   2%|▏         | 1/50 [00:03<03:10,  3.90s/it]LLM returned 1 generations instead of requested 3. Proceeding with 1 generations.
LLM returned 1 generations instead of requested 3. Proceeding with 1 generations.
Evaluating:  68%|██████▊   | 34/50 [01:55<00:48,  3.02s/it]LLM returned 1 generations instead of requested 3. Proceeding with 1 generations.
LLM returned 1 generations instead of requested 3. Proceeding with 1 generations.
Evaluating: 100%|██████████| 50/50 [02:32<00:00,  3.06s/it]



🔧 Running config: chunk=300, overlap=100, model=text-embedding-3-small, top_k=2
Split 48 pages into 620 chunks.

Saved 620 chunks to chroma.


Evaluating:   2%|▏         | 1/50 [00:04<03:16,  4.01s/it]LLM returned 1 generations instead of requested 3. Proceeding with 1 generations.
LLM returned 1 generations instead of requested 3. Proceeding with 1 generations.
Evaluating: 100%|██████████| 50/50 [02:51<00:00,  3.43s/it]



✅ Evaluation complete! Results saved to ragas_auto_eval_results.csv


,chunk_size,chunk_overlap,embedding_model,top_k,answer_correctness_mean,answer_correctness_std,answer_relevancy_mean,answer_relevancy_std,faithfulness_mean,faithfulness_std,context_precision_mean,context_precision_std,context_recall_mean,context_recall_std
0,300,0,text-embedding-3-small,2,0.451644,0.217702,0.960114,0.040333,0.516667,0.320156,0.7,0.458258,0.9,0.300000
1,300,50,text-embedding-3-small,2,0.505532,0.245905,0.878463,0.293491,0.561667,0.346093,0.7,0.458258,0.7,0.458258
2,300,100,text-embedding-3-small,2,0.410277,0.189378,0.983396,0.025447,0.441667,0.361421,0.7,0.458258,0.7,0.458258


In [ ]:
import itertools
import numpy as np
import pandas as pd
from ragas import evaluate
from ragas.metrics import (
    answer_correctness,
    answer_relevancy,
    faithfulness,
    context_precision,
    context_recall,
)
from datasets import Dataset
from IPython.display import display
# ============================================================================
# 🆕 NEW IMPORTS - Using Modern Libraries
# ============================================================================
import time
import evaluate as hf_evaluate  # Hugging Face evaluate library
from ranx import Qrels, Run, evaluate as ranx_evaluate

# Load metrics once (more efficient)
bleu_metric = hf_evaluate.load("bleu")
rouge_metric = hf_evaluate.load("rouge")
exact_match_metric = hf_evaluate.load("exact_match")
# BERTScore (optional - slower but more accurate)
try:
    bertscore_metric = hf_evaluate.load("bertscore")
    BERTSCORE_AVAILABLE = True
except:
    BERTSCORE_AVAILABLE = False
    print("⚠️  BERTScore not available. Install with: pip install bert-score")
# ============================================================================


# Load environment variables that contains the OpenAI API key
load_dotenv()

# Set path for where to get the original file and where to safe the chunks
CHROMA_PATH = "chroma"
DATA_PATH = "data/BoardGamesRuleBook"

# Example parameter grid
chunk_sizes = [300]
chunk_overlaps = [0, 50, 100]
embedding_models = ["text-embedding-3-small"]
retriever_top_k = [2]


# ============================================================================
# 🆕 SIMPLIFIED HELPER FUNCTIONS (Much Cleaner!)
# ============================================================================

def extract_doc_id_from_context(context_text):
    """
    Helper to extract a document identifier from context
    Using first 100 chars as unique identifier
    """
    return context_text[:100]


def identify_relevant_docs(contexts, reference_answer):
    """
    Identify which retrieved contexts are actually relevant
    Returns dict mapping query_id -> {doc_id: relevance_score}
    
    This is a heuristic - in production you'd want ground truth labels
    """
    relevant_scores = {}
    
    # Simple heuristic: TF-IDF style relevance
    ref_terms = set(reference_answer.lower().split())
    
    for ctx in contexts:
        ctx_terms = set(ctx.lower().split())
        # Overlap ratio as relevance score
        overlap = len(ref_terms & ctx_terms) / len(ref_terms) if ref_terms else 0
        
        doc_id = extract_doc_id_from_context(ctx)
        # Binary relevance: 1 if overlap > threshold, 0 otherwise
        relevant_scores[doc_id] = 1 if overlap > 0.2 else 0
    
    return relevant_scores


def calculate_retrieval_metrics(retrieved_contexts, reference_answer, k, query_id="q1"):
    """
    Calculate all retrieval metrics using ranx library
    
    Returns dict of metrics: precision@k, recall@k, mrr, ndcg@k, hit_rate@k
    """
    # Build ground truth relevance (qrels)
    # In production, you'd have pre-labeled relevant documents
    relevant_scores = identify_relevant_docs(retrieved_contexts, reference_answer)
    
    qrels_dict = {query_id: relevant_scores}
    qrels = Qrels(qrels_dict)
    
    # Build retrieval run results
    # Using position-based scores (higher rank = higher score)
    run_dict = {query_id: {}}
    for i, ctx in enumerate(retrieved_contexts):
        doc_id = extract_doc_id_from_context(ctx)
        # Score decreases with rank (1.0, 0.9, 0.8, ...)
        score = 1.0 - (i * 0.1)
        run_dict[query_id][doc_id] = score
    
    run = Run(run_dict)
    
    # Calculate all metrics at once using ranx
    metrics_to_calc = [
        f"precision@{k}",
        f"recall@{k}", 
        f"ndcg@{k}",
        "mrr",
        f"hit_rate@{k}"
    ]
    
    results = ranx_evaluate(qrels, run, metrics_to_calc)
    
    return results


def calculate_answer_metrics(reference, prediction):
    """
    Calculate all answer quality metrics using Hugging Face evaluate
    
    Returns dict with bleu, rouge, exact_match, and optionally bertscore
    """
    metrics = {}
    
    # BLEU score
    bleu_result = bleu_metric.compute(
        predictions=[prediction],
        references=[[reference]]
    )
    metrics['bleu'] = bleu_result['bleu']
    
    # ROUGE scores (L, 1, 2)
    rouge_result = rouge_metric.compute(
        predictions=[prediction],
        references=[reference]
    )
    metrics['rouge_l'] = rouge_result['rougeL']
    metrics['rouge_1'] = rouge_result['rouge1']
    metrics['rouge_2'] = rouge_result['rouge2']
    
    # Exact Match
    em_result = exact_match_metric.compute(
        predictions=[prediction],
        references=[reference]
    )
    metrics['exact_match'] = em_result['exact_match']
    
    # BERTScore (optional - slower but captures semantic similarity)
    if BERTSCORE_AVAILABLE:
        try:
            bert_result = bertscore_metric.compute(
                predictions=[prediction],
                references=[reference],
                lang="en"
            )
            metrics['bertscore_f1'] = bert_result['f1'][0]
        except:
            metrics['bertscore_f1'] = None
    
    # Simple token F1 (fast alternative to BERTScore)
    ref_tokens = set(reference.lower().split())
    pred_tokens = set(prediction.lower().split())
    
    if len(pred_tokens) > 0 and len(ref_tokens) > 0:
        common = ref_tokens & pred_tokens
        precision = len(common) / len(pred_tokens)
        recall = len(common) / len(ref_tokens)
        token_f1 = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0
    else:
        token_f1 = 0.0
    
    metrics['token_f1'] = token_f1
    
    return metrics

# ============================================================================
# END OF HELPER FUNCTIONS
# ============================================================================


results = []
documents = load_documents(DATA_PATH)

for chunk_size, overlap, emb_model, top_k in itertools.product(chunk_sizes, chunk_overlaps, embedding_models, retriever_top_k):
    print(f"\n🔧 Running config: chunk={chunk_size}, overlap={overlap}, model={emb_model}, top_k={top_k}")

    # --- Step 1: Build your retriever & RAG pipeline ---
    chunks = split_text(documents, chunk_size, overlap)
    db = save_to_chroma(chunks, CHROMA_PATH, emb_model)

    # generate answer in evaluation format and change it to dataset
    qa_file = "data/BoardGamesRuleBook/CATAN_Q&A_Eval.txt"
    
    # ========================================================================
    # 🆕 SIMPLIFIED EVALUATION LOOP
    # ========================================================================
    evaluation_rows = []
    
    # Aggregated metrics across all queries
    all_retrieval_metrics = []
    all_answer_metrics = []
    retrieval_times = []
    generation_times = []
    total_tokens = []
    
    # Parse Q&A file
    qa_rows = []
    current_game = None
    question = None

    with open(qa_file, 'r', encoding='utf-8') as f:
        for line in f:
            line = line.strip()
            if not line:
                continue
            
            if line.lower().startswith("board_game:"):
                current_game = line[len("board_game:"):].strip()
            elif line.lower().startswith("q:"):
                question = line[len("q:"):].strip()
            elif line.lower().startswith("a:"):
                answer = line[len("a:"):].strip()
                if current_game is None:
                    raise ValueError(f"Answer found without specifying board game: {line}")
                if question is None:
                    raise ValueError(f"Answer found without a question: {line}")
                qa_rows.append({
                    "board_game": current_game,
                    "question": question,
                    "reference": answer
                })
                question = None
    
    # Process each Q&A pair
    for i, row in enumerate(qa_rows):
        query = row["question"]
        ground_truth = row["reference"]
        board_game = row["board_game"]
        query_id = f"q{i+1}"
        
        # Measure retrieval time
        retrieval_start = time.time()
        context = db.similarity_search_with_relevance_scores(query, k=top_k)
        retrieval_time = time.time() - retrieval_start
        retrieval_times.append(retrieval_time)
        
        # Extract contexts
        contexts = [doc.page_content for doc, _score in context]
        
        # Measure generation time
        generation_start = time.time()
        answer = generate_answer(query, context)
        generation_time = time.time() - generation_start
        generation_times.append(generation_time)
        
        # Track tokens (approximate)
        tokens = sum(len(ctx.split()) for ctx in contexts) + len(answer.content.split())
        total_tokens.append(tokens)
        
        # 🆕 Calculate retrieval metrics using ranx (one line!)
        retrieval_metrics = calculate_retrieval_metrics(contexts, ground_truth, top_k, query_id)
        all_retrieval_metrics.append(retrieval_metrics)
        
        # 🆕 Calculate answer metrics using HF evaluate (one line!)
        answer_metrics = calculate_answer_metrics(ground_truth, answer.content)
        all_answer_metrics.append(answer_metrics)
        
        # Add to evaluation dataset for RAGAs
        evaluation_rows.append({
            "question": query,
            "contexts": contexts,
            "answer": answer.content,
            "reference": ground_truth,
            "source": board_game
        })
    
    evaluation_dataset = Dataset.from_list(evaluation_rows)
    # ========================================================================
    # END OF EVALUATION LOOP
    # ========================================================================

    # sync interface compatible with RAGas
    llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)

    # Run RAGAs evaluation
    scores = evaluate(
        evaluation_dataset,
        metrics=[
            answer_correctness,
            answer_relevancy,
            faithfulness,
            context_precision,
            context_recall,
        ],
        llm=llm,
    )

    # ========================================================================
    # 🆕 AGGREGATE METRICS (Much Cleaner!)
    # ========================================================================
    
    # Helper to safely get metric values
    def safe_mean(metric_list, key):
        values = [m.get(key, 0) for m in metric_list if m.get(key) is not None]
        return np.mean(values) if values else 0.0
    
    def safe_std(metric_list, key):
        values = [m.get(key, 0) for m in metric_list if m.get(key) is not None]
        return np.std(values) if values else 0.0
    
    results.append({
        # Configuration
        "chunk_size": chunk_size,
        "chunk_overlap": overlap,
        "embedding_model": emb_model,
        "top_k": top_k,
        
        # RAGAs metrics (existing)
        "answer_correctness_mean": np.mean(scores["answer_correctness"]),
        "answer_correctness_std": np.std(scores["answer_correctness"]),
        "answer_relevancy_mean": np.mean(scores["answer_relevancy"]),
        "answer_relevancy_std": np.std(scores["answer_relevancy"]),
        "faithfulness_mean": np.mean(scores["faithfulness"]),
        "faithfulness_std": np.std(scores["faithfulness"]),
        "context_precision_mean": np.mean(scores["context_precision"]),
        "context_precision_std": np.std(scores["context_precision"]),
        "context_recall_mean": np.mean(scores["context_recall"]),
        "context_recall_std": np.std(scores["context_recall"]),
        
        # 🆕 Retrieval metrics (from ranx)
        f"precision@{top_k}_mean": safe_mean(all_retrieval_metrics, f"precision@{top_k}"),
        f"precision@{top_k}_std": safe_std(all_retrieval_metrics, f"precision@{top_k}"),
        f"recall@{top_k}_mean": safe_mean(all_retrieval_metrics, f"recall@{top_k}"),
        f"recall@{top_k}_std": safe_std(all_retrieval_metrics, f"recall@{top_k}"),
        "mrr_mean": safe_mean(all_retrieval_metrics, "mrr"),
        "mrr_std": safe_std(all_retrieval_metrics, "mrr"),
        f"hit_rate@{top_k}_mean": safe_mean(all_retrieval_metrics, f"hit_rate@{top_k}"),
        f"hit_rate@{top_k}_std": safe_std(all_retrieval_metrics, f"hit_rate@{top_k}"),
        f"ndcg@{top_k}_mean": safe_mean(all_retrieval_metrics, f"ndcg@{top_k}"),
        f"ndcg@{top_k}_std": safe_std(all_retrieval_metrics, f"ndcg@{top_k}"),
        
        # 🆕 Answer quality metrics (from HF evaluate)
        "bleu_mean": safe_mean(all_answer_metrics, "bleu"),
        "bleu_std": safe_std(all_answer_metrics, "bleu"),
        "rouge_l_mean": safe_mean(all_answer_metrics, "rouge_l"),
        "rouge_l_std": safe_std(all_answer_metrics, "rouge_l"),
        "rouge_1_mean": safe_mean(all_answer_metrics, "rouge_1"),
        "rouge_2_mean": safe_mean(all_answer_metrics, "rouge_2"),
        "token_f1_mean": safe_mean(all_answer_metrics, "token_f1"),
        "token_f1_std": safe_std(all_answer_metrics, "token_f1"),
        "exact_match_rate": safe_mean(all_answer_metrics, "exact_match"),
        "bertscore_f1_mean": safe_mean(all_answer_metrics, "bertscore_f1") if BERTSCORE_AVAILABLE else None,
        
        # 🆕 Performance metrics
        "avg_retrieval_time_ms": np.mean(retrieval_times) * 1000,
        "avg_generation_time_ms": np.mean(generation_times) * 1000,
        "avg_total_time_ms": np.mean([r + g for r, g in zip(retrieval_times, generation_times)]) * 1000,
        "avg_tokens_per_query": np.mean(total_tokens),
        "total_queries": len(qa_rows),
    })
    # ========================================================================
    # END OF METRICS COLLECTION
    # ========================================================================

# --- Step 4: Save and inspect ---
df = pd.DataFrame(results)
df.to_csv("ragas_auto_eval_results_enhanced.csv", index=False)
print("\n✅ Evaluation complete! Results saved to ragas_auto_eval_results_enhanced.csv")

# ============================================================================
# 🆕 ENHANCED DISPLAY
# ============================================================================
print("\n" + "="*80)
print("📊 EVALUATION RESULTS SUMMARY")
print("="*80)

# Display key metrics grouped by category
print("\n🎯 RAGAS Semantic Metrics:")
ragas_cols = ["chunk_size", "chunk_overlap", "top_k",
              "answer_correctness_mean", "answer_relevancy_mean", 
              "faithfulness_mean", "context_precision_mean", "context_recall_mean"]
display(df[ragas_cols].round(3))

print("\n🔍 Retrieval Metrics (ranx):")
retrieval_cols = ["chunk_size", "chunk_overlap", "top_k",
                  f"precision@{retriever_top_k[0]}_mean", 
                  f"recall@{retriever_top_k[0]}_mean",
                  "mrr_mean", f"ndcg@{retriever_top_k[0]}_mean",
                  f"hit_rate@{retriever_top_k[0]}_mean"]
display(df[retrieval_cols].round(3))

print("\n📝 Answer Quality Metrics (HF evaluate):")
answer_cols = ["chunk_size", "chunk_overlap", "top_k",
               "bleu_mean", "rouge_l_mean", "rouge_1_mean",
               "token_f1_mean", "exact_match_rate"]
display(df[answer_cols].round(3))

print("\n⏱️  Performance Metrics:")
perf_cols = ["chunk_size", "chunk_overlap", "avg_retrieval_time_ms", 
             "avg_generation_time_ms", "avg_total_time_ms", "avg_tokens_per_query"]
display(df[perf_cols].round(2))

print("\n" + "="*80)
print("💡 Installation reminder:")
print("   pip install evaluate ranx bert-score nltk rouge-score")
print("="*80)
# ============================================================================

In [ ]:
import itertools
import numpy as np
import pandas as pd
from ragas import evaluate
from ragas.metrics import (
    answer_correctness,
    answer_relevancy,
    faithfulness,
    context_precision,
    context_recall,
)
from datasets import Dataset
from IPython.display import display
from dotenv import load_dotenv

# Hugging Face retrieval metrics
import evaluate as hf_evaluate

# Load environment variables (OpenAI API key)
load_dotenv()

# Paths
CHROMA_PATH = "chroma"
DATA_PATH = "data/BoardGamesRuleBook"  # LLMs paper

# Parameter grid
chunk_sizes = [300]
chunk_overlaps = [0, 50, 100]
embedding_models = ["text-embedding-3-small"]
retriever_top_k = [2]

# Load dataset and documents
documents = load_documents(DATA_PATH)

# Initialize Hugging Face retrieval metrics
hf_metrics = {
    "precision": hf_evaluate.load("precision", module_type="metric"),
    "recall": hf_evaluate.load("recall", module_type="metric"),
    "mrr": hf_evaluate.load("mrr", module_type="metric"),
    "ndcg": hf_evaluate.load("ndcg", module_type="metric"),
    "hit_rate": hf_evaluate.load("hit_rate", module_type="metric"),
}

results = []

for chunk_size, overlap, emb_model, top_k in itertools.product(
    chunk_sizes, chunk_overlaps, embedding_models, retriever_top_k
):
    print(f"\n🔧 Running config: chunk={chunk_size}, overlap={overlap}, model={emb_model}, top_k={top_k}")

    # --- Step 1: Build your retriever & RAG pipeline ---
    chunks = split_text(documents, chunk_size, overlap)
    db = save_to_chroma(chunks, CHROMA_PATH, emb_model)

    # --- Step 2: Prepare evaluation dataset ---
    qa_file = "data/BoardGamesRuleBook/CATAN_Q&A_Eval.txt"
    evaluation_rows = generate_answer_in_eval_format(qa_file, db, top_k)
    evaluation_dataset = Dataset.from_list(evaluation_rows)

    # RAGas evaluation
    llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)
    ragas_scores = evaluate(
        evaluation_dataset,
        metrics=[
            answer_correctness,
            answer_relevancy,
            faithfulness,
            context_precision,
            context_recall,
        ],
        llm=llm,
    )

    # --- Step 3: Hugging Face retrieval evaluation ---
    # HF retrieval metrics require:
    # - predictions: list of lists of top-k document ids
    # - references: list of lists of relevant document ids
    predictions = [row["retrieved_chunk_ids"] for row in evaluation_rows]
    references = [row["relevant_chunk_ids"] for row in evaluation_rows]

    hf_scores = {}
    for name, metric in hf_metrics.items():
        hf_scores[name] = metric.compute(predictions=predictions, references=references, k=top_k)

    # --- Step 4: Collect results ---
    results.append({
        "chunk_size": chunk_size,
        "chunk_overlap": overlap,
        "embedding_model": emb_model,
        "top_k": top_k,
        # RAGas metrics
        "answer_correctness_mean": np.mean(ragas_scores["answer_correctness"]),
        "answer_correctness_std": np.std(ragas_scores["answer_correctness"]),
        "answer_relevancy_mean": np.mean(ragas_scores["answer_relevancy"]),
        "answer_relevancy_std": np.std(ragas_scores["answer_relevancy"]),
        "faithfulness_mean": np.mean(ragas_scores["faithfulness"]),
        "faithfulness_std": np.std(ragas_scores["faithfulness"]),
        "context_precision_mean": np.mean(ragas_scores["context_precision"]),
        "context_precision_std": np.std(ragas_scores["context_precision"]),
        "context_recall_mean": np.mean(ragas_scores["context_recall"]),
        "context_recall_std": np.std(ragas_scores["context_recall"]),
        # HF retrieval metrics
        "precision_at_k": hf_scores["precision"]["precision"],
        "recall_at_k": hf_scores["recall"]["recall"],
        "mrr_at_k": hf_scores["mrr"]["mrr"],
        "ndcg_at_k": hf_scores["ndcg"]["ndcg"],
        "hit_rate_at_k": hf_scores["hit_rate"]["hit_rate"],
    })

# --- Step 5: Save results ---
df = pd.DataFrame(results)
df.to_csv("ragas_hf_eval_results.csv", index=False)
print("\n✅ Evaluation complete! Results saved to ragas_hf_eval_results.csv")
display(df)

Reference<br>
[How to Evaluate Retrieval Quality in RAG Pipelines (part 2): Mean Reciprocal Rank (MRR) and Average Precision (AP)](https://towardsdatascience.com/how-to-evaluate-retrieval-quality-in-rag-pipelines-part-2-mean-reciprocal-rank-mrr-and-average-precision-ap/)<br>
[How to Evaluate Retrieval Quality in RAG Pipelines: Precision@k, Recall@k, and F1@k](https://towardsdatascience.com/how-to-evaluate-retrieval-quality-in-rag-pipelines-precisionk-recallk-and-f1k/)<br>
[RAG Evaluation: Don’t let customers tell you first](https://www.pinecone.io/learn/series/vector-databases-in-production-for-busy-engineers/rag-evaluation/)<br>
[Demystifying NDCG](https://medium.com/data-science/demystifying-ndcg-bee3be58cfe0)<br>